# KAGGLE

In [9]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [10]:
df = pd.read_csv('df_train.csv')
df

,id,LoanNr_ChkDgt,Name,City,State,Bank,BankState,ApprovalDate,ApprovalFY,NoEmp,...,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,Accept
0,bd9d6267ec5,1523195006,"P-SCAPE LAND DESIGN, LLC",NORTHFIELD,OH,CITIZENS BANK NATL ASSOC,RI,2005-11-01,2006,2,...,0,2,0,1,0.0,0.0,2005-12-31,8000.0,0.0,1
1,9eebf6d8098,1326365010,The Fresh & Healthy Catering C,CANTON,OH,"FIRSTMERIT BANK, N.A.",OH,2005-06-06,2005,2,...,1,2,1,1,0.0,0.0,2005-07-31,166000.0,0.0,1
2,83806858500,6179584001,AARON MASON & HOWE LLC,SAWYERWOOD,OH,"PNC BANK, NATIONAL ASSOCIATION",OH,2003-03-18,2003,2,...,4,2,1,2,1.0,0.0,2003-03-31,25000.0,0.0,1
3,a21ab9cb3af,8463493009,MID OHIO CAR WASH,COLUMBUS,OH,THE HUNTINGTON NATIONAL BANK,OH,1995-06-28,1995,2,...,0,0,1,0,0.0,0.0,1996-01-31,220100.0,0.0,1
4,883b5e5385e,3382225007,Bake N Brew LLC,Newark,OH,THE HUNTINGTON NATIONAL BANK,OH,2009-04-16,2009,0,...,0,0,0,1,0.0,0.0,2009-05-31,25000.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22830,4f9443d2a46,1573725008,"SIBILA RACE ENGINEERING, INC",MASSILLON,OH,CITIZENS BANK NATL ASSOC,RI,2005-12-09,2006,1,...,0,1,0,1,0.0,0.0,2005-12-31,70000.0,0.0,1
22831,798db2753a7,2011184008,ENVIRO SHIELD POWER WASHING,SPRINGBORO,OH,"PNC BANK, NATIONAL ASSOCIATION",OH,1998-04-27,1998,2,...,0,0,1,0,0.0,1.0,1998-05-31,30000.0,0.0,1
22832,ddb3c5e9bff,4082983001,"MAINLINE TRCK&TRAILR SRVC, INC",BEDFORD,OH,GROWTH CAPITAL CORP.,OH,1990-05-09,1990,16,...,6,10,1,0,0.0,0.0,1991-02-13,92000.0,0.0,1
22833,407200a5dfe,7783283010,TIN BOX STUDIO,CINCINNATI,OH,KEYBANK NATIONAL ASSOCIATION,OH,1994-11-10,1995,1,...,0,0,1,0,0.0,1.0,1995-01-31,20000.0,0.0,1


In [11]:
# 2. Separar clases mayoritaria y minoritaria
df_majority = df[df["Accept"] == 1]  # Créditos aceptados (clase mayoritaria)
df_minority = df[df["Accept"] == 0]  # Créditos no aceptados (clase minoritaria)

In [12]:
# 3. Aplicar Submuestreo (undersampling)
df_majority_downsampled = resample(df_majority, 
                                   replace=False,  # Sin reemplazo
                                   n_samples=len(df_minority),  # Igualar cantidad de muestras
                                   random_state=42)  # Fijar semilla

In [13]:
# 4. Unir el dataset balanceado
df_balanced = pd.concat([df_majority_downsampled, df_minority])

In [14]:
# 5. Mezclar los datos para evitar sesgos
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

In [15]:
# Eliminar filas con NaN en las columnas relevantes
columnas_relevantes = ['City', 'NoEmp', 'UrbanRural', 'NewExist', 'Accept']
df_balanced= df_balanced.dropna(subset=columnas_relevantes)
df_balanced = df_balanced.drop(columns=["id"])
df_balanced = df_balanced.drop(columns=["LoanNr_ChkDgt"])
df_balanced = df_balanced.drop(columns=["Name"])
df_balanced = df_balanced.drop(columns=["ApprovalDate"])
df_balanced = df_balanced.drop(columns=["DisbursementDate"])
df_balanced = df_balanced.drop(columns=["State"])

df_balanced.head()

,City,Bank,BankState,ApprovalFY,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,Accept
0,Greenwillow,THE HUNTINGTON NATIONAL BANK,OH,2007,5,1.0,0,1,0,1,0.0,0.0,31000.0,0.0,0
1,COLUMBUS,JPMORGAN CHASE BANK NATL ASSOC,IL,2008,7,1.0,0,7,0,1,1.0,0.0,61461.0,0.0,1
2,COLUMBUS,THE HUNTINGTON NATIONAL BANK,OH,2008,4,0.0,0,4,1,1,0.0,0.0,50000.0,0.0,1
3,Reynoldsburg,"BUSINESS LOAN CENTER, LLC",FL,2006,3,1.0,0,3,0,1,0.0,0.0,25000.0,0.0,0
4,WESTLAKE,CITIZENS BANK NATL ASSOC,RI,2006,2,0.0,0,2,0,1,0.0,0.0,42000.0,0.0,1


In [16]:
# Seleccionar variables predictoras y la variable objetivo "Accept"
X = df_balanced[['City', 'NoEmp', 'UrbanRural', 'NewExist']] 
y = df_balanced['Accept']

# Convertir variables categóricas a dummies (City, State y UrbanRural)
X = pd.get_dummies(X, columns=['City', 'UrbanRural'], drop_first=True)

joblib.dump(X.columns.tolist(), "features_list.pkl")

df_balanced

,City,Bank,BankState,ApprovalFY,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,BalanceGross,Accept
0,Greenwillow,THE HUNTINGTON NATIONAL BANK,OH,2007,5,1.0,0,1,0,1,0.0,0.0,31000.0,0.0,0
1,COLUMBUS,JPMORGAN CHASE BANK NATL ASSOC,IL,2008,7,1.0,0,7,0,1,1.0,0.0,61461.0,0.0,1
2,COLUMBUS,THE HUNTINGTON NATIONAL BANK,OH,2008,4,0.0,0,4,1,1,0.0,0.0,50000.0,0.0,1
3,Reynoldsburg,"BUSINESS LOAN CENTER, LLC",FL,2006,3,1.0,0,3,0,1,0.0,0.0,25000.0,0.0,0
4,WESTLAKE,CITIZENS BANK NATL ASSOC,RI,2006,2,0.0,0,2,0,1,0.0,0.0,42000.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7657,Norton,"PNC BANK, NATIONAL ASSOCIATION",DE,2007,2,0.0,0,2,0,2,0.0,0.0,35400.0,0.0,0
7658,Powell,THE HUNTINGTON NATIONAL BANK,OH,2007,0,0.0,0,0,0,2,0.0,0.0,25000.0,0.0,0
7659,CLEVES,KEYBANK NATIONAL ASSOCIATION,OH,2006,4,1.0,0,4,0,1,1.0,0.0,56000.0,0.0,1
7660,NEW LEXINGTON,THE PEOPLES NATIONAL BANK,OH,1983,5,1.0,0,0,0,0,0.0,0.0,190000.0,0.0,0


In [17]:
# 2. Dividir en Train y Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=42)

In [18]:
# Normalizar características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [19]:
# 3. Definir el modelo
model = LogisticRegression(max_iter=1000)

In [20]:
# 4. Entrenar el modelo
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [21]:
# 5. Evaluar el modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.6105


In [22]:
# 6. Matriz de confusión y F-score
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", classification_report(y_test, y_pred))

Confusion Matrix:
 [[1622  666]
 [1124 1184]]
Classification Report:
               precision    recall  f1-score   support

           0       0.59      0.71      0.64      2288
           1       0.64      0.51      0.57      2308

    accuracy                           0.61      4596
   macro avg       0.62      0.61      0.61      4596
weighted avg       0.62      0.61      0.61      4596



In [23]:
import joblib
import datetime
import pandas as pd

# Cargar el test
df_test = pd.read_csv("df_test_nolabel.csv")

# Mismo preprocesamiento
cols_to_drop = ['id', 'LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'DisbursementDate', 'State']
df_test_clean = df_test.drop(columns=cols_to_drop)

# Llenar nulos y seleccionar variables
df_test_clean.fillna(0, inplace=True)
X_test = df_test_clean[['Bank', 'City', 'BankState', 'NoEmp', 'UrbanRural', 'NewExist', 'DisbursementGross']]

# One-hot encoding
X_test = pd.get_dummies(X_test, columns=['Bank', 'City', 'BankState', 'UrbanRural'], drop_first=True)

# Cargar las features esperadas
features = joblib.load("features_list.pkl")

# Reindexar con esas columnas
X_test = X_test.reindex(columns=features, fill_value=0)

# Predecir
df_test['Accept'] = model.predict(X_test)
df_test['Accept'] = df_test['Accept'].astype(int)

# --- Exportar Submission ---
filename = f"regresion_logistica_{datetime.datetime.now().strftime('%Y%m%d')}.csv"
df_test.to_csv(filename, columns=['id', 'Accept'], index=False)

print(f"✅ Archivo de submission '{filename}' generado correctamente.")


✅ Archivo de submission 'regresion_logistica_20250404.csv' generado correctamente.
